<a href="https://colab.research.google.com/github/jibarletta/PAECIS/blob/main/Codigo_Reddit_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ReadMe: A modo de introducción a las explicaciones que encontrarán en este código, nos parece relevante aclarar que, luego de pruebas y modificaciones hemos decidido armar un sólo dataset que agrupe los 10 post más votados de cada día y todos sus correspondientes comentarios.
Se scrapea esta información por partes. Primero los temas o posts (los cuales antes eran separados en un data set aparte) y posteriormente todos los comentarios de ese tema. Luego hemos entendido que por operatoria convenía juntarnos en un gran dataset para su posterior análisis.

El código puede ser observado en el siguiente [enlace](https://github.com/jibarletta/mokugeki/blob/master/main.py).

Primero importamos las librerías necesarias.

In [ ]:
import pandas as pd
import datetime
from datetime import datetime as dt
from credenciales import redditauth
from pathlib import Path

ModuleNotFoundError: No module named 'credenciales'

Luego validamos las credenciales

In [ ]:
reddit = redditauth()

NameError: name 'redditauth' is not defined

A continuación se genera un datestamp para su posterior uso. Será muy necesario para clasificar datos y archivos.

In [ ]:
filename_date = dt.now().strftime('%Y-%m-%d')  # -%H-%M removido del filename

Primero armamos un diccionario con los 10 posts más populares del subreddit de "Argentina".

In [ ]:
post_dict = {"id": [],
             "context": [],
             "text": [],
             "upv": [],
             "author": [],
             "date": [],
             "type": [],
             }
arg_subreddit = reddit.subreddit('Argentina')
for post in arg_subreddit.top(limit=10, time_filter='day'):
    post_dict["id"].append(post.id)
    post_dict["context"].append(post.id)
    post_dict["text"].append(post.title)
    post_dict["upv"].append(post.score)
    post_dict["author"].append(post.author)
    post_dict["date"].append(filename_date)
    post_dict['type'].append('post')

NameError: name 'reddit' is not defined

Se convierte el diccionario en un dataframe. Se lo guarda en CSV. Luego se lo formatea para concatenarlo con los posteriores datos que se sacarán del subreddit. También se arma un csv aparte sólo con los 10 temas del día.

In [ ]:
# Se convierte el diccionario en un data frame.
temas = pd.DataFrame(post_dict)
temas.to_csv(f"{Path.cwd()}/Bases/Temas/Reddit_Arg_Temas{filename_date}.csv")
dataset = pd.read_csv(f"{Path.cwd()}/Bases/Dataset/Red_Arg_Dataset.csv", index_col=0)
nuevods = pd.concat([dataset, temas], sort=False)
nuevods.reset_index(inplace=True, drop=True)
nuevods.to_csv(f"{Path.cwd()}/Bases/Dataset/Red_Arg_Dataset.csv")

NameError: name 'Path' is not defined

Se realiza el mismo procedimiento para los comentarios de cada uno de los post del diccionario anterior temas. Se concatena al dataset general.

In [ ]:
# Mismo procedimiento anterior pero para los comentarios de cada post.
lista_ids = temas["id"]
for postid in lista_ids:
    comment_dict = {"id": [],
                    "context": [],
                    "text": [],
                    "upv": [],
                    "author": [],
                    "date": [],
                    "type": [],
                    }
    posteo = reddit.submission(postid)
    posteo.comments.replace_more(limit=0)
    for comentario in posteo.comments.list():
        if comentario.author == 'empleadoEstatalBot':  # Elimina el bot que postea la noticia en caso de existir.
            continue
        comment_dict["id"].append(comentario.id)
        p_split = comentario.parent_id.split('_')
        idctx = p_split[1]
        comment_dict['context'].append(idctx)
        comment_dict["text"].append(comentario.body)
        comment_dict["upv"].append(comentario.score)
        comment_dict["author"].append(comentario.author)
        comment_dict['date'].append(filename_date)
        comment_dict['type'].append('comment')

En esta última sección de termina de concatenar el dataset final y se separa aparte un archivo sólo con los comentarios del correpsondiente post.

In [ ]:
    dataset = pd.read_csv(f"{Path.cwd()}/Bases/Dataset/Red_Arg_Dataset.csv", index_col=0)
    comms = pd.DataFrame(comment_dict)
    nuevods = pd.concat([dataset, comms], sort=False)
    nuevods.reset_index(inplace=True, drop=True)
    nuevods.to_csv(f"{Path.cwd()}/Bases/Dataset/Red_Arg_Dataset.csv")
    comms.to_csv(f"{Path.cwd()}/Bases/Comentarios/Red_Arg_Comm_Post-{postid}-{filename_date}.csv")
    archivo = f"Red_Arg_Comm_Post-{postid}-{filename_date}.csv"